In [ ]:
from stitching import Stitcher
import cv2 as cv

settings = {"detector": "orb", "range_width":200 ,"confidence_threshold": 0.01, "print_time": True, "crop": False}
stitcher = Stitcher(**settings)

settings1 = {"detector": "sift", "range_width":900 ,"confidence_threshold": 0.01, "print_time": True, "crop": False}
stitcher1 = Stitcher(**settings1)
panorama0 = stitcher.stitch([cv.imread("image1.jpg"), cv.imread("image2.jpg")])
panorama1 = stitcher.stitch([cv.imread("image3.jpg"), cv.imread("image4.jpg")])
panorama = stitcher1.stitch([panorama0, panorama1])
# cv.imwrite("panorama.jpg", panorama)  # save panorama

In [7]:


import cv2 as cv
from multiprocessing import Pool, cpu_count
from time import time
from stitching import Stitcher

# 封装图像拼接函数
def stitch_images(image_paths, settings, settings1):
    # 使用 settings 进行初步拼接
    stitcher = Stitcher(**settings)
    panoramas = []
    for i in range(0, len(image_paths), 2):
        images = [cv.imread(image_paths[i]), cv.imread(image_paths[i + 1])]
        panorama = stitcher.stitch(images)
        panoramas.append(panorama)
    
    # 使用 settings1 进行最终拼接
    stitcher1 = Stitcher(**settings1)
    final_panorama = stitcher1.stitch(panoramas)
    return final_panorama

# 多进程处理函数
def multiprocess_stitch(image_path_groups, settings, settings1, num_processes):
    with Pool(num_processes) as pool:
        panoramas = pool.starmap(stitch_images, [(group, settings, settings1) for group in image_path_groups])
    return panoramas

if __name__ == "__main__":
    # 定义图像路径组（在测试中，可以使用同样的源进行多次拼接）
    image_path_groups = [
        ["image1.jpg", "image2.jpg", "image3.jpg", "image4.jpg"] for _ in range(8)
    ]

    # 设置参数
    settings = {"detector": "orb", "range_width": 200, "confidence_threshold": 0.01, "print_time": False, "crop": False}
    settings1 = {"detector": "sift", "range_width": 900, "confidence_threshold": 0.01, "print_time": False, "crop": False}

    # 测试性能
    num_processes = max(8, cpu_count())  # 使用最多8个进程

    start_time = time()
    panoramas = multiprocess_stitch(image_path_groups, settings, settings1, num_processes)
    
    # 保存最终拼接图像
    for idx, panorama in enumerate(panoramas):
        cv.imwrite(f"panorama_{idx}.jpg", panorama)
    
    end_time = time()

    print(f"Total stitching time with {num_processes} processes: {end_time - start_time:.4f} seconds")
    print(f"fps : {num_processes/ (end_time-start_time):.3f} ")


Total stitching time with 16 processes: 1.0924 seconds
fps : 14.647 
